In [1]:
import pandas as pd
#from datetime import date, time
DelDF = pd.read_csv('Deliveries.csv')
OldDelDF = pd.read_csv('DeliveriesOld.csv')
DaysDF = pd.read_csv('Days.csv')
WeeksDF = pd.read_csv('Weeks.csv')
gpsDF = pd.read_csv('GPS.csv')
rawDF = pd.read_csv('RawData.csv')

In [2]:
#For every attribute x from columns, replace $ and , with empty, then convert to float
#Should we do for index, row then df.at[index, x] = float()
#Or just pop it from the for loop?

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)

        
#For every attribute x from columns, for every row convert hh:mm string to double

def time_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)
            
            
#This obviously only works for 2023
#Might want to homogenize to columns just in case it shows up later but for now only use is 1 column

def date_to_timestamp(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
        
        
#This only works for 2023, but also orders that end on the same date as shift start
#So if an order finishes at 1am it wont work
#Need to update with conditional like if end is AM start is PM then date +=1

def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [3]:
frames = [OldDelDF, DelDF]
result = pd.concat(frames)
#We need to reset the index after the combination of v1/v2, and v3
result.reset_index(drop=True, inplace=True)

#DelDF is v3
#results is all deliveries
#Those are the only 2 we need

In [4]:
result.dtypes

ID                  int64
Date               object
Restaurant Name    object
Total              object
Pay                object
Tip                object
Peak Bonus         object
Stacked            object
Version             int64
Start Time         object
Miles              object
End Time           object
Est. Duration      object
dtype: object

In [5]:
dollars_to_floats(result, ["Total", "Pay", "Tip", "Peak Bonus"])
dollars_to_floats(OldDelDF, ["Total", "Pay", "Tip", "Peak Bonus"])

/tmp/ipykernel_46674/53385023.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [6]:
#Doodle to get the percentage of orders that tip
numUntipped = (result["Tip"] == 0.0).sum()
numTipped = (result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 3)
print(percTipped*100, "% of customers tip")

89.3 % of customers tip


In [7]:
# GPS DOODLE HERE
# Initial doodle to help with address entry
# Need to rework to handle both CSVs now
#gpsDF1 = gpsDF[["ID", "Date", "Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF1[["Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF2.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
#gpsDF2 = gpsDF2.sort_values(by="Restaurant Name", ascending=True)
#gpsDF2.to_csv('gpsDF2.csv')

In [8]:
tempDF = result.copy()
tempDF = tempDF[result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Pay": "mean", "Tip": "mean"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF.sort_values("Tip", ascending=False)

,Pay,Tip
Restaurant Name,,
Albertsons,4.58,12.39
Hunter Steakhouse,10.00,11.50
Chin's Szechwan,5.88,9.25
Mangia e Bevi,6.25,9.00
Maans Mediterranean Grill and Kabobery,5.75,7.70
...,...,...
Lollicup Fresh,4.25,1.82
Carl's Jr.,4.38,1.50
Sizzler,12.75,1.25


In [9]:
#Overall hourly rate, Total from TOTALS / Dash
#Hardcoded index for Totals, should look for row where dates == "TOTALS" 

#Find row where Dates == "Totals"
#tempDF that row
#dollars_to_floats
#time_to_floats
#hourlyRate = tempDF["Total"]/tempDF["Dash"]
#print("$", hourlyRate)

tempDF = WeeksDF.copy()
#double bracket on iloc returns a df instead of series
tempDF = tempDF.iloc[[-1]]
dollars_to_floats(tempDF, ["Total"])
time_to_floats(tempDF, ["Dash"])
hourlyRate = tempDF["Total"]/tempDF["Dash"]
print(tempDF["Total"])
print(tempDF["Dash"])
float(tempDF["Dash"])
#print("$", hourlyRate)

<class 'float'>
12    7112.19
Name: Total, dtype: float64
12    320.3
Name: Dash, dtype: float64


/tmp/ipykernel_46674/53385023.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


320.3

In [10]:
#So float(var) converts, but series.astype(float) is for series in df

In [11]:
#Doodle to 
#convert $#.## to float and hh:mm:ss to float, 
#then make the pre total to hours worked ratio, then sort and show

print(WeeksDF.dtypes)

dollars_to_floats(WeeksDF, ["Pre Total"])
time_to_floats(WeeksDF, ["Active"])

WeeksDF["pretotalactiveratio"] = WeeksDF["Pre Total"]/WeeksDF["Active"]
WeeksDF.sort_values(by=["pretotalactiveratio"])
#pre total is the money before adjustment, its just base pay + tip 

#Trying to maximize total, so I guess the question really is which factor or ratio of factors
#contributes the most

Dates         object
Base          object
Tip           object
Adj pay       object
Other         object
Pre Total     object
Total         object
Active        object
Dash          object
Deliveries     int64
Version       object
dtype: object
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


/tmp/ipykernel_46674/53385023.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version,pretotalactiveratio
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,226.88,$287.95,8.63,11:10:00,28,1,26.289687
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,729.73,$871.11,25.50,41:10:00,84,1,28.616863
12,TOTALS:,"$3,544.75","$2,714.44",$844.13,$8.87,6259.19,"$7,112.19",216.62,320:18:00,674,-,28.894793
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,196.50,$237.46,6.70,10:55:00,22,1,29.328358
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,630.91,$735.96,21.33,32:18:00,68,1,29.578528
7,5/15 - 5/21,$303.75,$239.25,$99.27,$0.00,543.00,$642.27,17.53,25:24:00,64,2,30.975471
10,6/5 - 6/11,$197.25,$185.09,$65.38,$0.00,382.34,$447.72,11.77,15:45:00,41,3,32.484282
6,5/8 - 5/14,$758.25,$567.84,$155.48,$6.25,1326.09,"$1,487.82",40.58,58:08:00,138,2,32.678413
5,5/1 - 5/7,$618.50,$445.32,$121.74,$0.00,1063.82,"$1,185.56",32.40,50:40:00,114,1,32.833951
9,5/29 - 6/4,$342.75,$309.88,$115.30,$0.00,652.63,$767.93,19.40,26:32:00,63,3,33.640722


In [12]:
#Dollars Earned vs Miles Driven doodle
#$ to float
#drop non numeric Miles values, then convert column to float
#agg sum Total and Miles, by restaurant name (then round)
#create new column as Total/Miles
#sort low to high
tempDF = DelDF.copy()
dollars_to_floats(tempDF, ["Total"])
tempDF = tempDF[tempDF.Miles!="-"]
tempDF["Miles"] = tempDF["Miles"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Miles": "sum"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Miles"]
tempDF.sort_values(by="DollarMilesRatio")

/tmp/ipykernel_46674/53385023.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


,Total,Miles,DollarMilesRatio
Restaurant Name,,,
Pedro's Tacos,7.00,7.9,0.886076
Anita's Mexican Restaurant & Cantina,4.75,4.9,0.969388
Urban Plates,21.50,20.4,1.053922
Lollicup Fresh,6.25,5.6,1.116071
Colimas Mexican Food,6.25,5.3,1.179245
...,...,...,...
Cafe de Thai & Sushi,21.75,5.7,3.815789
Albertsons,34.17,8.8,3.882955
Graziano's Piza,9.50,2.3,4.130435


In [13]:
DaysDF.max() #would be useful with more attributes converted to float 

Date                    6/9
Total                $86.13
Base                 $99.75
Tip                  $94.03
Start (24)            18:58
End (24)               2:32
Active time (h:m)      9:40
Dash time (h:m)        9:41
Offers                   32
Deliveries               33
Mileage Start         69045
Mileage End           69192
Version                   3
Pauses                    2
dtype: object

In [14]:
#Avg tip vs Avg tip excluding zero tip orders

TipAvg = result["Tip"].mean()
print(TipAvg)
TipAvg2 = result[result["Tip"] !=0]["Tip"].mean()
print(TipAvg2)

4.076875
4.567092846270929


In [15]:
#CONVERT STRING OF DATE TO TIMESTAMP, THEN TIMESTAMP TO DATETIME
date_to_timestamp(DaysDF, "Date")
DaysDF["Date"] = pd.to_datetime(DaysDF["Date"])

In [16]:
#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
DaysDF["Day of Week"] = DaysDF["Date"].dt.day_name()

In [17]:
time_to_floats(DaysDF, ["Dash time (h:m)"])
dollars_to_floats(DaysDF, ["Total", "Base", "Tip"])

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


/tmp/ipykernel_46674/53385023.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [18]:
DaysDF = DaysDF.groupby(["Day of Week"]).agg(
    {"Dash time (h:m)": "sum", "Total": "sum", "Tip": "sum", "Deliveries ": "sum"})
DaysDF = DaysDF.round({'Dash time (h:m)': 2, 'Total': 2, "Tip": 2})
DaysDF["$/hr"] = DaysDF["Total"]/DaysDF["Dash time (h:m)"]
DaysDF["AvgTip"] = DaysDF["Tip"]/DaysDF["Deliveries "]
display(DaysDF)

,Dash time (h:m),Total,Tip,Deliveries,$/hr,AvgTip
Day of Week,,,,,,
Friday,94.85,2082.03,940.78,221,21.950764,4.256923
Monday,7.02,135.97,56.97,14,19.368946,4.069286
Saturday,75.89,1744.22,782.72,178,22.983529,4.397303
Sunday,37.63,882.78,408.03,90,23.459474,4.533667
Thursday,56.86,1167.71,484.21,134,20.536581,3.613507
Wednesday,48.00,845.87,327.87,99,17.622292,3.311818
